# This is a simple introduction to use of Gurobi MILP solver.

Consider the following linear problem
$$\begin{array}{ll}
\max &4x_1+5x_2\\
&-6x_1+10x_2\le 25\\
&2x_1+x_2\le 16\\
&x_2\le 5\\
&x_1,x_2\ge 0
\end{array}
$$


## We can obtaint the optimal solution graphically
<img src="Gurobi01.jpg" alt="Drawing" style="width: 600px;"/>


In [10]:
# setup the Gurobi environment
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# create your model
m = gp.Model("example01_a")
m.reset()

In [4]:
# Create variables
x1 = m.addVar(vtype=GRB.CONTINUOUS, name="x1")
x2 = m.addVar(vtype=GRB.CONTINUOUS, name="x2")

In [5]:
# Set objective
m.setObjective(4*x1 + 5*x2, GRB.MAXIMIZE)

In [ ]:
# Add constraints
m.addConstr(-6*x1+10*x2 <= 25, "c1")
m.addConstr(2*x1+x2 <= 16, "c2")
m.addConstr(x2 <= 5, "c3")


In [ ]:
# Optimize model
m.optimize()

In [ ]:
# Output results
print('Obj: %g' % m.objVal)

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))



In [9]:
# writing your model to a text file
m.write("example01_a.lp")

\ Model example01<br>
\ LP format - for model browsing. Use MPS format to capture full model detail.<br>
Maximize<br>
&nbsp;&nbsp;&nbsp;  4 x1 + 5 x2<br>
Subject To<br>
&nbsp;&nbsp;&nbsp;c1: - 6 x1 + 10 x2 <= 25<br>
&nbsp;&nbsp;&nbsp;c2: 2 x1 + x2 <= 16<br>
&nbsp;&nbsp;&nbsp;c3: x2 <= 5<br>
Bounds<br>
Generals<br>
 x1 x2<br>
End<br>

In [10]:
# Switching to an integer program (x1, x2 must assume integer values)
x1.setAttr(GRB.Attr.VType, GRB.INTEGER)
x2.setAttr(GRB.Attr.VType, GRB.INTEGER)

In [ ]:
# Optimize MIP model
m.optimize()

In [ ]:
# Output MIP results
print('Obj: %g' % m.objVal)

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))



## Revise the above model by considering to represent the data with lists (sets, arrays,..)

$$\begin{array}{rrcr}
\max &4x_1&+5x_2\\
&-6x_1&+10x_2&\le &25\\
&2x_1&+x_2&\le &16\\
&&x_2&\le &5\\
&x_1,&x_2&\ge &0
\end{array}
$$

$$
\begin{array}{|rr|r|}
\hline4&5\\
\hline
-6&10& 25\\
2&1&16\\
0&1&5\\
\hline
\end{array}
$$


In [ ]:
# Using lists and arrays
m2 = gp.Model("Exercise01_b")
m2.reset()
variableNames = ['x1','x2']
c = [4,5]
A = {
      (0,0): -6, (0,1): 10,
      (1,0):  2, (1,1):  1,
      (2,0):  0, (2,1):  1
}
b = [25, 16, 5]

In [20]:
# add variables with names given in 'varNames' 
x = m2.addVars(range(len(variableNames)), vtype=GRB.CONTINUOUS, name=variableNames)

In [21]:
# add objective function
m2.setObjective(x.prod(c), GRB.MAXIMIZE)

In [22]:
# add constraints
# corrisponde alla sommatoria da j[0:n-1] di A[i][j]*x[j]<=b[i] per i[0:m-1] dove n è il numero di colonne e m il numero di righe
m2.addConstrs( gp.quicksum(A[i,j] * x[j] for j in range(len(x)) ) <= b[i] for i in range(len(b)) )

m2.write("Example01_b.lp")

In [ ]:
m2.optimize()

In [ ]:
# Output
print('Obj: %g' % m2.objVal)

for v in m2.getVars():
    print('%s %g' % (v.varName, v.x))
    
for c in m2.getConstrs():
    print('slack= %g  rhs= %g' % (c.Slack, c.RHS))


In [ ]:
for v in m2.getVars(): 
    v.setAttr(GRB.Attr.VType, GRB.INTEGER)

In [ ]:
m2.write("Example01_c.lp")

In [ ]:
m2.optimize()